In [ ]:
import pandas as pd
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import de la basse de données
path = "/content/drive/MyDrive/Openclassrooms/Projet 3/DAN-P3-data.csv"
df_raw = pd.read_csv(path, decimal=",", parse_dates=['Date mutation'])
df_raw.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (20,26,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Code type de voie,Type de voie,Code voie,Voie,Code ID commune,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,No Volume,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,3eme lot,Surface Carrez du 3eme lot,4eme lot,Surface Carrez du 4eme lot,5eme lot,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-02-03,Vente,56000.0,190.0,A,0,RUE,5,CENTRALE,0,1370.0,SAINT-ETIENNE-DU-BOIS,1,350,NaN,B,1478,NaN,22,50.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,Appartement,NaN,52,2,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,165000.0,347.0,NaN,0,RUE,20,DU CHATEAU,1,1170.0,CHEVRY,1,103,NaN,A,302,NaN,12,48.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,48,3,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-08,Vente,720000.0,58.0,NaN,1,AV,527,DU MONT BLANC,2,1220.0,DIVONNE-LES-BAINS,1,143,NaN,AK,563,NaN,146,130.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,130,6,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-06,Vente,429250.0,140.0,NaN,0,RUE,2,DE L'ABBE JOLIVET,3,1630.0,PERON,1,288,NaN,C,2307,NaN,11,109.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,Maison,NaN,109,5,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-07,Vente,220900.0,39.0,NaN,0,RUE,110,BUFFON,4,1200.0,VALSERHONE,1,33,NaN,AE,440,NaN,31,108.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,Appartement,NaN,91,4,NaN,NaN,NaN


## Analyse rapide du dataframe

In [ ]:
# Copie du dataframe original
df = df_raw.copy()

In [ ]:
# Liste des colonnes vides
cols_vides = [col for col in df.columns if df[col].isnull().all()]
cols_vides

['Code service CH',
 'Reference document',
 '1 Articles CGI',
 '2 Articles CGI',
 '3 Articles CGI',
 '4 Articles CGI',
 '5 Articles CGI',
 'No Volume',
 '2eme lot',
 'Surface Carrez du 2eme lot',
 '3eme lot',
 'Surface Carrez du 3eme lot',
 '4eme lot',
 'Surface Carrez du 4eme lot',
 '5eme lot',
 'Surface Carrez du 5eme lot',
 'Identifiant local']

In [ ]:
# Suppression des colonnes vides
df.drop(cols_vides, axis=1, inplace=True)

In [ ]:
# Aucun doublon
df.duplicated().sum()

0

In [ ]:
# Vérification des valeurs manquantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34169 entries, 0 to 34168
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   No disposition             34169 non-null  int64         
 1   Date mutation              34169 non-null  datetime64[ns]
 2   Nature mutation            34169 non-null  object        
 3   Valeur fonciere            34151 non-null  float64       
 4   No voie                    34036 non-null  float64       
 5   B/T/Q                      2174 non-null   object        
 6   Code type de voie          34169 non-null  int64         
 7   Type de voie               33229 non-null  object        
 8   Code voie                  34169 non-null  object        
 9   Voie                       34169 non-null  object        
 10  Code ID commune            34169 non-null  int64         
 11  Code postal                34168 non-null  object        
 12  Comm

Beaucoup de valeurs manquantes. Rien d'anormal sauf pour "Valeur fonciere" qui pourrait fausser les calculs de prix au m²

In [ ]:
# Suppression des valeurs nulles de "Valeur fonciere"
df.dropna(subset=['Valeur fonciere'], inplace=True)

## Préparation des tables pour l'import SQL

#### Table commune  
id (PK), code_postal, nom_commune, code_dpt

In [ ]:
df_commune = df[["Code ID commune", "Code postal", "Commune", "Code departement"]]

In [ ]:
# Renommage des colonnes pour l'import mySQL
df_commune.columns = ["id", "code_postal", "nom_commune", "code_dpt"]


In [ ]:
# Création d'un id unique
df_commune.drop_duplicates(subset="id", inplace=True)
df_commune.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3215 entries, 0 to 33925
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           3215 non-null   int64 
 1   code_postal  3214 non-null   object
 2   nom_commune  3215 non-null   object
 3   code_dpt     3215 non-null   object
dtypes: int64(1), object(3)
memory usage: 125.6+ KB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Création du fichier "commune.csv" pour l'import mySQL
df_commune.to_csv("commune.csv", index=False)

#### Table bien

In [ ]:
# Création du dataframe
df_bien = df[["Type local", "Nombre pieces principales",
              "Surface Carrez du 1er lot",
              "Code ID commune"]].copy()
df_bien.head()


,Type local,Nombre pieces principales,Surface Carrez du 1er lot,Code ID commune
0,Appartement,2,50.42,0
1,Appartement,3,48.22,1
2,Appartement,6,130.80,2
3,Maison,5,109.22,3
4,Appartement,4,108.65,4


In [ ]:
# Création de l'id bien (composite par rapport au références cadastrales + N° du lot 1)
df_bien["id"] = df["Code departement"].astype("str") + df["Code commune"].astype("str") \
    + df["Prefixe de section"].astype("str") + df["Section"] + df["No plan"].astype("str") \
    + df["1er lot"].astype("str")


In [ ]:
df_bien

,Type local,Nombre pieces principales,Surface Carrez du 1er lot,Code ID commune,id
0,Appartement,2,50.42,0,1350nanB147822
1,Appartement,3,48.22,1,1103nanA30212
2,Appartement,6,130.80,2,1143nanAK563146
3,Maison,5,109.22,3,1288nanC230711
4,Appartement,4,108.65,4,133nanAE44031
...,...,...,...,...,...
34164,Appartement,3,102.09,3213,75104nanAN3113
34165,Appartement,3,55.95,3213,75104nanAG7967
34166,Appartement,2,32.86,3213,75104nanAO7322
34167,Appartement,3,56.80,3213,75104nanAM6789


In [ ]:
df_bien.columns = ["type_local", "nb_piece", "surf_carrez", "commune_id", "id"]
df_bien.head()

,type_local,nb_piece,surf_carrez,commune_id,id
0,Appartement,2,50.42,0,1350nanB147822
1,Appartement,3,48.22,1,1103nanA30212
2,Appartement,6,130.80,2,1143nanAK563146
3,Maison,5,109.22,3,1288nanC230711
4,Appartement,4,108.65,4,133nanAE44031


In [ ]:
df_bien.drop_duplicates(subset="id", inplace=True)

In [ ]:
df_bien = df_bien.reindex(columns=["id", "type_local", "nb_piece", "surf_carrez", "commune_id"])



In [ ]:
df_bien["surf_carrez"].astype("float64")

0         50.42
1         48.22
2        130.80
3        109.22
4        108.65
          ...  
34164    102.09
34165     55.95
34166     32.86
34167     56.80
34168     35.44
Name: surf_carrez, Length: 34142, dtype: float64

In [ ]:
df_bien.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34142 entries, 0 to 34168
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           34141 non-null  object 
 1   type_local   34142 non-null  object 
 2   nb_piece     34142 non-null  int64  
 3   surf_carrez  34142 non-null  float64
 4   commune_id   34142 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.6+ MB


In [ ]:
df_bien.to_csv("bien.csv", index=False)

## Table mutation

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34151 entries, 0 to 34168
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   No disposition             34151 non-null  int64         
 1   Date mutation              34151 non-null  datetime64[ns]
 2   Nature mutation            34151 non-null  object        
 3   Valeur fonciere            34151 non-null  float64       
 4   No voie                    34018 non-null  float64       
 5   B/T/Q                      2173 non-null   object        
 6   Code type de voie          34151 non-null  int64         
 7   Type de voie               33211 non-null  object        
 8   Code voie                  34151 non-null  object        
 9   Voie                       34151 non-null  object        
 10  Code ID commune            34151 non-null  int64         
 11  Code postal                34150 non-null  object        
 12  Comm

In [ ]:
df.head()

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Code type de voie,Type de voie,Code voie,Voie,Code ID commune,Code postal,Commune,Code departement,Code commune,Prefixe de section,Section,No plan,1er lot,Surface Carrez du 1er lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,1,2020-02-03,Vente,56000.0,190.0,A,0,RUE,5,CENTRALE,0,1370.0,SAINT-ETIENNE-DU-BOIS,1,350,NaN,B,1478,22,50.42,1,2,Appartement,52,2,NaN,NaN,NaN
1,1,2020-01-02,Vente,165000.0,347.0,NaN,0,RUE,20,DU CHATEAU,1,1170.0,CHEVRY,1,103,NaN,A,302,12,48.22,2,2,Appartement,48,3,NaN,NaN,NaN
2,1,2020-01-08,Vente,720000.0,58.0,NaN,1,AV,527,DU MONT BLANC,2,1220.0,DIVONNE-LES-BAINS,1,143,NaN,AK,563,146,130.80,2,2,Appartement,130,6,NaN,NaN,NaN
3,1,2020-01-06,Vente,429250.0,140.0,NaN,0,RUE,2,DE L'ABBE JOLIVET,3,1630.0,PERON,1,288,NaN,C,2307,11,109.22,1,1,Maison,109,5,NaN,NaN,NaN
4,1,2020-01-07,Vente,220900.0,39.0,NaN,0,RUE,110,BUFFON,4,1200.0,VALSERHONE,1,33,NaN,AE,440,31,108.65,2,2,Appartement,91,4,NaN,NaN,NaN


In [ ]:
df_mutation = df[["Date mutation", "Valeur fonciere"]]

In [ ]:
df_mutation.columns = ["date_mut", "prix_mut"]

,date_mut,prix_mut
0,2020-02-03,56000.0
1,2020-01-02,165000.0
2,2020-01-08,720000.0
3,2020-01-06,429250.0
4,2020-01-07,220900.0
...,...,...
34164,2020-06-30,1290000.0
34165,2020-06-30,726000.0
34166,2020-06-25,420000.0
34167,2020-06-25,725000.0


In [ ]:
df_mutation["bien_id"] = df["Code departement"].astype("str") + df["Code commune"].astype("str") \
    + df["Prefixe de section"].astype("str") + df["Section"] + df["No plan"].astype("str") \
    + df["1er lot"].astype("str")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_mutation.index = df_mutation.index + 1

In [ ]:
df_mutation.to_csv("mutation.csv")